In [4]:
import scipy.io

import pandas as pd, numpy as np

pd.set_option('display.max_rows', None)  # Set the option to display all rows

data = scipy.io.loadmat("/Users/leondeligny/Desktop/PDM/PINN/Raissi/cylinder_nektar_wake.mat")

U_star = data['U_star'] # N x 2 x T
p_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2

# Averaging U, V and p over the time dimension
U_mean = U_star.mean(axis=2)[:, 0]  # Averaging over time, selecting U component
V_mean = U_star.mean(axis=2)[:, 1]  # Averaging over time, selecting V component
p_mean = p_star.mean(axis=1)  # Average pressure over time

# Create a DataFrame with averaged values
df_train = pd.DataFrame({
    'x': X_star[:, 0],
    'y': X_star[:, 1],
    'u': U_mean,
    'v': V_mean,
    'p': p_mean
})

x = np.linspace(-15.0, 10.0, 2501)
y = np.linspace(-6.0, 6.0, 1201)
xx, yy = np.meshgrid(x, y)
grid = np.column_stack([xx.ravel(), yy.ravel()])
df = pd.DataFrame(grid, columns=['x', 'y'])

# Assigning 'sdf' value based on the condition of being on the circle
df['sdf'] = np.sqrt(df['x']**2 + df['y']**2) - 0.5

df.loc[df['x'] == -15.0, 'u'] = 1
df.loc[df['x'] == -15.0, 'v'] = 0
df.loc[df['sdf'] <= 0.0, 'u'] = 0
df.loc[df['sdf'] <= 0.0, 'v'] = 0

# Save the DataFrame
df.to_csv('Raissi_dataset.csv', index=False)

            x     y           sdf    u    v
1377050  0.00 -0.50  0.000000e+00  0.0  0.0
1379542 -0.09 -0.49 -1.803252e-03  0.0  0.0
1379543 -0.08 -0.49 -3.512337e-03  0.0  0.0
1379544 -0.07 -0.49 -5.025253e-03  0.0  0.0
1379545 -0.06 -0.49 -6.340198e-03  0.0  0.0
1379546 -0.05 -0.49 -7.455586e-03  0.0  0.0
1379547 -0.04 -0.49 -8.370058e-03  0.0  0.0
1379548 -0.03 -0.49 -9.082492e-03  0.0  0.0
1379549 -0.02 -0.49 -9.592007e-03  0.0  0.0
1379550 -0.01 -0.49 -9.897970e-03  0.0  0.0
1379551  0.00 -0.49 -1.000000e-02  0.0  0.0
1379552  0.01 -0.49 -9.897970e-03  0.0  0.0
1379553  0.02 -0.49 -9.592007e-03  0.0  0.0
1379554  0.03 -0.49 -9.082492e-03  0.0  0.0
1379555  0.04 -0.49 -8.370058e-03  0.0  0.0
1379556  0.05 -0.49 -7.455586e-03  0.0  0.0
1379557  0.06 -0.49 -6.340198e-03  0.0  0.0
1379558  0.07 -0.49 -5.025253e-03  0.0  0.0
1379559  0.08 -0.49 -3.512337e-03  0.0  0.0
1379560  0.09 -0.49 -1.803252e-03  0.0  0.0
1382038 -0.14 -0.48 -2.775558e-16  0.0  0.0
1382039 -0.13 -0.48 -2.707330e-0

In [11]:
import torch

# Load the DataFrame
df = pd.read_csv('Raissi_dataset.csv')

mask_inlet = df['x'] == -15.0
mask_sdf = df['sdf'] <= 0.0
mask_top = df['y'] == 6.0
mask_bot = df['y'] == -6.0


from PINN_Raissi import PINN_Raissi

# Train the model
model = PINN_Raissi(df_train, mask_inlet, mask_sdf, mask_top, mask_bot)
    
# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

print(f"Started Training.")
model.train(501)
print(f"Finished Training.")

             x    y        sdf    u   v
3001200 -15.00  6.0  15.655494  1.0 NaN
3001201 -14.99  6.0  15.646210  NaN NaN
3001202 -14.98  6.0  15.636927  NaN NaN
3001203 -14.97  6.0  15.627644  NaN NaN
3001204 -14.96  6.0  15.618362  NaN NaN
3001205 -14.95  6.0  15.609081  NaN NaN
3001206 -14.94  6.0  15.599801  NaN NaN
3001207 -14.93  6.0  15.590522  NaN NaN
3001208 -14.92  6.0  15.581244  NaN NaN
3001209 -14.91  6.0  15.571966  NaN NaN
3001210 -14.90  6.0  15.562690  NaN NaN
3001211 -14.89  6.0  15.553414  NaN NaN
3001212 -14.88  6.0  15.544139  NaN NaN
3001213 -14.87  6.0  15.534865  NaN NaN
3001214 -14.86  6.0  15.525592  NaN NaN
3001215 -14.85  6.0  15.516320  NaN NaN
3001216 -14.84  6.0  15.507048  NaN NaN
3001217 -14.83  6.0  15.497778  NaN NaN
3001218 -14.82  6.0  15.488508  NaN NaN
3001219 -14.81  6.0  15.479240  NaN NaN
3001220 -14.80  6.0  15.469972  NaN NaN
3001221 -14.79  6.0  15.460705  NaN NaN
3001222 -14.78  6.0  15.451439  NaN NaN
3001223 -14.77  6.0  15.442174  NaN NaN


'\nfrom PINN_Raissi import PINN_Raissi\n\n# Train the model\nmodel = PINN_Raissi(df_train, mask_inlet, mask_sdf, mask_upper, mask_bottom)\n    \n# Check for CUDA availability\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n\nmodel.to(device)\n\nprint(f"Started Training.")\nmodel.train(501)\nprint(f"Finished Training.")\n'

In [ ]:
from plot import plot_predictions_vs_test

# Prediction u_pred, v_pred, p_pred, nut_pred
u_pred, v_pred, p_pred = model.predict(df_train)

# Plotting
plot_predictions_vs_test(df_train['x'].astype(float).values.flatten(), df_train['y'].astype(float).values.flatten(), u_pred, df_train['u'], 'u', 'PINN_Raissi_FourierF')
plot_predictions_vs_test(df_train['x'].astype(float).values.flatten(), df_train['y'].astype(float).values.flatten(), v_pred, df_train['v'], 'v', 'PINN_Raissi_FourierF')
plot_predictions_vs_test(df_train['x'].astype(float).values.flatten(), df_train['y'].astype(float).values.flatten(), p_pred, df_train['p'], 'p', 'PINN_Raissi_FourierF')